In [1]:
import numpy as np
import pandas as pd
import scipy.linalg as LA
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.image as mpimg
from PIL import Image
from scipy.misc import imread
from scipy.stats import skew
from scipy.stats.stats import pearsonr
from sklearn.linear_model import Ridge, LassoCV, Lasso
import xgboost as xgb
from sklearn.linear_model import LogisticRegression

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

all_data = pd.concat((train.loc[:,'RESOURCE':'ROLE_CODE'],
                      test.loc[:,'RESOURCE':'ROLE_CODE']))

#matplotlib.rcParams['figure.figsize'] = (12.0, 6.0)
#prices = pd.DataFrame({"price":train["SalePrice"], "log(price + 1)":np.log1p(train["SalePrice"])})
#prices.hist()
#log transform the target:
train["Action"] = np.log1p(train["ACTION"])

#log transform skewed numeric features:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index
                               
skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

all_data = pd.get_dummies(all_data)

#filling NA's with the mean of the column:
all_data = all_data.fillna(all_data.mean())

#creating matrices for sklearn:
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = train.ACTION

In [3]:
X_train.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,10.580353,11.355991,117961,118300,123472,11.677643,11.677651,290919,11.677668
1,9.751734,7.340187,117961,118343,123125,11.682980,11.682980,308574,11.683006
2,10.511213,9.579003,118219,118220,117884,11.677422,12.498567,19721,11.677431
3,10.495045,8.593599,117961,118343,119993,11.681165,12.392486,290919,11.681173
4,10.661509,8.683724,117929,117930,119569,11.689598,11.727496,19793,11.689615


In [4]:
X_test.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,11.274249,11.194578,118079,118080,117878,11.677422,11.679947,19721,11.677431
1,10.612631,8.384576,117961,118327,118507,11.685735,11.711850,118398,11.685752
2,11.231146,7.781556,117961,118300,119488,11.679905,12.616641,249618,11.679930
3,10.674059,9.902837,117961,118225,118403,11.701676,11.821792,118960,11.701685
4,10.647660,10.820098,117961,118343,119598,11.682018,12.611994,118424,11.682044


In [61]:
y.shape

(32769,)

In [23]:
log = LogisticRegression(solver='lbfgs')
log = log.fit(X_train,y)

In [24]:
predictions = log.predict(X_test)

In [25]:
results = pd.DataFrame({"Id":test.id, "Action":predictions})
results.to_csv("LogisticResults.csv", index = False)

In [26]:
predictions

array([1, 1, 1, ..., 1, 1, 1])

In [20]:
log.score(X_train, y)

0.94210992096188473

In [ ]:
model_xgb = xgb.XGBClassifier(n_estimators=1000, max_depth=10, learning_rate=0.6, objective='binary:logistic')
model_xgb.fit(X_train, y)

In [75]:
xgb_preds = model_xgb.predict(X_test)

results = pd.DataFrame({"id":test.id})
results["ACTION"]=xgb_preds
results.to_csv("XGBresults1.csv", index = False)

In [76]:
results.head()

,id,ACTION
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1


In [78]:
model_xgb.evals_result()

AttributeError: 'XGBClassifier' object has no attribute 'evals_result'